In [2]:
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
}
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0'}

In [3]:
prices = []
names = []
powers = []
gears = []
carbus = []

info_dic = {"Type": [], "Modèle": [], "Marque": [], "Kilométrage": [], "État": [], "Première main": [],
            "Origine": [], "Année-Modèle": [], "Nombre de portes": [], "Secteur": []}
keys = list(info_dic.keys())
headers = {'User-Agent': 'Mozilla/5.0'}
try:
    for i in range(1, 2500):
        linkPrincipal = "https://www.avito.ma/fr/maroc/voitures-%C3%A0_vendre?o="
        linkPrincipal = linkPrincipal + str(i)
        response = None
        while response is None or response.status_code != 200:
            response = requests.get(linkPrincipal, headers=headers)
            time.sleep(random.uniform(1, 3))
        soup = BeautifulSoup(response.content, "html.parser")

        car_divs = soup.find("div", {"class": "sc-1nre5ec-1 fzpnun listing"})
        if car_divs:
            cars = car_divs.find_all("div", {"class": ["sc-jejop8-0 epNjzr", "sc-jejop8-0 kyKuDw"]})

            links = [car.find("a")["href"] for car in cars]

            if i % 20 == 0:
                df = pd.DataFrame(info_dic)
                df.to_csv(f"cars_at_page_{i}.csv")
                
            for link in links:
                response = requests.get(link, headers=headers)
                time.sleep(random.uniform(1, 3))
                soup = BeautifulSoup(response.content, "html.parser")
                name = soup.find("div", {"class": "sc-1g3sn3w-9 gIlAYt"}).text
                price = soup.find("div", {"class": "sc-1g3sn3w-10 iMiMHk"}).text
                prices.append(price)
                names.append(name)
                infos = soup.find_all("span", {"class": "sc-1x0vz2r-0 kuCwGF"})

                power_found = False
                gear_found = False
                fuel_found = False

                for info in infos:
                    text = info.text
                    if text in ['Automatique', "Manuelle"]:
                        gears.append(text)
                        gear_found = True
                    elif text.isalpha() == False:
                        powers.append(text)
                        power_found = True
                    else:
                        carbus.append(text)
                        fuel_found = True

                if not power_found:
                    powers.append(np.nan)
                if not gear_found:
                    gears.append(np.nan)
                if not fuel_found:
                    carbus.append(np.nan)

                infos_page = soup.find("ol", {"class": "sc-qmn92k-3"})
                infos_data = infos_page.find_all("li", {"class": "sc-qmn92k-1 ldnQxr"})
                existed = []

                for info in infos_data:
                    key = info.find("span", {"class": "sc-1x0vz2r-0 brylYP"}).text
                    existed.append(key)
                    info_dic[key].append(info.find("span", {"class": "sc-1x0vz2r-0 jsrimE"}).text)

                missing = list(set(keys) - set(existed))
                for mis in missing:
                    info_dic[mis].append(np.nan)

                print(link, "Page : " + str(i))
            info_dic["Name"] = names
            info_dic["Price"] = prices
            info_dic["Power"] = powers
            info_dic["Gear"] = gears
            info_dic["Fuel"] = carbus
except:
    df = pd.DataFrame(info_dic)
    df.to_csv(f"cars_at_page_{i}.csv")

        


https://www.avito.ma/fr/hassan/voitures/BMW_serie_520d_Diesel_2015_51703464.htm Page : 1
https://www.avito.ma/fr/maarif/voitures/Mercedes_Benz_classe_gle_350d_pack_AMG_Diesel_2017_50746967.htm Page : 1
https://www.avito.ma/fr/autre_secteur/voitures/Honda_CRV__50407822.htm Page : 1
https://www.avito.ma/fr/casablanca/voitures/Hummer_H3__52484147.htm Page : 1
https://www.avito.ma/fr/casablanca/voitures/Chrysler_Grand_voyager_limited_52483814.htm Page : 1
https://www.avito.ma/fr/casablanca/voitures/Maserati_quattroporte_f1_49281600.htm Page : 1
https://www.avito.ma/fr/casablanca/voitures/tucson_top_52735304.htm Page : 1
https://www.avito.ma/fr/sidi_bernoussi/voitures/Peugeot_301_30000km_52533087.htm Page : 1
https://www.avito.ma/fr/autre_secteur/voitures/_AUDI_Q5_____52746324.htm Page : 1
https://www.avito.ma/fr/agadir/voitures/jeep_model_2021_Bon_état__52661494.htm Page : 1
https://www.avito.ma/fr/tanger/voitures/Citroen_C4_52698443.htm Page : 1
https://www.avito.ma/fr/souissi/voitures/Ra

In [ ]:
info_dic

In [ ]:
for key, value in info_dic.items():
    print(f"{key}: {len(value)}")